Se leen los ficheros de datos en el grid de 5 km de Aemet y se salvan en ficheros parquet

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq

Primero hay que definir el path de los ficheros gz con los datos de la rejilla en el diccionario files.

Después, tienes que asignar un valor válido a la variable var (son valore válidos 'pcp', 'tmax' o 'tmin')

In [14]:
# 'pcp, 'tmin, 'tmax'
var = 'tmax'

files = {
    'pcp': r'E:\DB\aemet\rejilla5km\Serie_AEMET_v2_pcp_1951a2022_txt.tar\pcp_red_SPAIN_1951-20221231.txt.gz',
    'tmax' : r'E:\DB\aemet\rejilla5km\Serie_AEMET_v1_tmax_1951a2022_txt.tar\tmax_red_SPAIN_1951-2022.geo.gz',
    'tmin' : r'E:\DB\aemet\rejilla5km\Serie_AEMET_v1_tmin_1951a2022_txt.tar\tmin_red_SPAIN_1951-2022.geo.gz'
}

if var not in files.keys():
    raise ValueError('Error al asignar var')

if files['pcp'] == files['tmax'] or files['pcp'] == files['tmin'] or files['tmax'] == files['tmin']:
    raise ValueError('No pude haber valores repetidos en files')

if var not in files[var]:
    raise ValueError(f'El fichero de Aemet debe contener {var}')

dir_out = Path(r'E:\DB\aemet\rejilla5km\parquet')
fopath = dir_out.joinpath(f"{var}.parquet")

if not dir_out.exists() or not dir_out.is_dir():
    raise ValueError(f'El directorio de salida no existe')


Se leen los ficheros gz y se guardan en formato parquet

In [15]:
df = pd.read_csv(files[var], nrows=5)
ncols = df.shape[1]
df = pd.read_csv(files[var], compression='gzip', header = None, sep=r'\s+', 
                 dtype={0: 'str', **{col: 'float32' for col in range(1, ncols)}})
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format='%Y%m%d')

table = pa.Table.from_pandas(df)
pq.write_table(table, fopath)


Rejilla

La rejilla en fpath se basa en la original de AEMET pero tiene 2 columns más: X e Y con las coordenadas de los puntos en EPSG 25830

In [4]:
fpath = r'E:\DB\aemet\rejilla5km\parquet\rejilla_25830.csv'

df = pd.read_csv(fpath, dtype={0: 'float32', 1: 'float32', 2: 'int32', 3: 'float32', 4: 'float32', 5: 'float32'})

df=df[['id', 'lon', 'lat', 'z', 'X', 'Y']]                 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16156 entries, 0 to 16155
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      16156 non-null  int32  
 1   lon     16156 non-null  float32
 2   lat     16156 non-null  float32
 3   z       16156 non-null  float32
 4   X       16156 non-null  float32
 5   Y       16156 non-null  float32
dtypes: float32(5), int32(1)
memory usage: 378.8 KB


In [5]:
df['epsg'] = '25830'

In [6]:
df = df.rename(columns = {'X':'x', 'Y':'y'})

In [7]:
fopath = r'E:\DB\aemet\rejilla5km\parquet\rejilla.parquet'
table = pa.Table.from_pandas(df)
pq.write_table(table, fopath)
